# Credit Card Fraud Detection 

In [1]:
%pylab inline
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

import pandas as pd
import numpy as np

np.random.seed(0)

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Data

[Credit Card Fraud Detection - Kaggle](https://www.kaggle.com/dalpozz/creditcardfraud)에서 다운받을수 있습니다.

데이터는 2013년 유럽 카드회사의 이틀동안 일어난 transactions에 관한 것이며, <br>
492건의 frauds 가 284,807건의 transactions중에 일어 났습니다.

Class에서 1은 fraud를 뜻하며, 0은 아닌것을 말합니다.

Time데이터는 첫번재 Column으로부터 몇초 이후에 발생한 transaction이라는 뜻입니다. <br>
나머지 데이터들은 PCA의 규제에 의해서 어떤 데이터인지 밝히지 않습니다.

In [53]:
data = pd.read_csv('/dataset/time-series/credit-card-fraud-detection/creditcard.csv')

# Preprocessing Amount
amt_scale = StandardScaler()
data['NormAmount'] =  amt_scale.fit_transform(data['Amount'])

# Split Train and Test Data
X = data.drop(['Time', 'Amount', 'Class'], axis=1).as_matrix()
Y = data['Class'].as_matrix()

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.25, random_state=1)

fraud_test_y = test_y == 1
fraud_test_x = test_x[fraud_test_y]
fraud_test_y = test_y[fraud_test_y]

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


#### Checking the number of fraud transactions in training and test data

In [35]:
print('The number of Fraud transactions in Training Data:', train_y[train_y == 1].shape[0])
print('The number of Fraud transactions in Test Data:',  test_y[test_y == 1].shape[0])

The number of Fraud transactions in Training Data: 381
The number of Fraud transactions in Test Data: 111


#### Checking the target classes

fraud transactions이 492개밖에 되지 않기 때문에, 일반적인 classification algorithm으로 돌리면 물론 정확도는 매우 높게 나오지만.. 
실상은 1에 해당하는 fraud transactions에서는 대부분 틀릴 가능성이 매우 높습니다. 


In [51]:
pd.value_counts(data['Class'], sort=True)

0    284315
1       492
Name: Class, dtype: int64

## Resampling

resampling에는 여러가지 방법이 있습니다. 

1. Over Sampling: SMOTE (Synthetic Minority Over-Sampling Technique)
2. Under Sampling

아래의 resample function에서는 5:5의 비율로 under sampling을 해줍니다.<br>
resample을 하면서 시간관계가 어차피 깨지기 때문에 (사실 각각의 transactions들 사이에 상관관계가 있는지도 모르겠음)<br>
shuffle을 통해서 train되는 데이터를 augment해줍니다.

In [49]:
def resample(X, Y):
    index = np.arange(Y.shape[0])
    fraud_indices = index[Y == 1]
    normal_indices = index[Y == 0]
    random_normal_indices = np.random.choice(normal_indices, len(fraud_indices))
    
    sample_indices = np.concatenate([fraud_indices, random_normal_indices])
    np.random.shuffle(sample_indices)
    sample_indices = np.array(sample_indices)
    
    sample_x = X[sample_indices]
    sample_y = Y[sample_indices]
    return sample_x, sample_y

## Logistic Regression

전체적으로 0.99% accuracy를 보이지만, 실제 fraud data만 test를 했을때는 0.57%로.. 실질적으로 못맞추는 수준입니다.<br>
사실 일반적인 알고리즘으로 학습시키기 위해서는 over sampling (SMOTE 같은) 또는 under sampling이 필요합니다.<br>
sampling을 통해서 skewed data를 보정하는 것입니다.

#### resample 없이 데이터 학습뒤 예측하면..

In [52]:
lg = LogisticRegression()
lg.fit(train_x, train_y)
predicted_y = lg.predict(test_x)
accuracy_score(test_y, predicted_y)

0.99914328249206485

In [55]:
predicted_y = lg.predict(fraud_test_x)
accuracy_score(fraud_test_y, predicted_y)

0.57657657657657657

#### resampled data로 학습뒤 예측하면...
0.88 ~ 0.92 까지 예측률이 높아지게 됩니다.

In [90]:
lg = LogisticRegression()
lg.fit(*resample(train_x, train_y))

predicted_y = lg.predict(test_x)
accuracy_score(test_y, predicted_y)

0.96132131119912367

In [91]:
predicted_y = lg.predict(fraud_test_x)
accuracy_score(fraud_test_y, predicted_y)

0.91891891891891897